This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# Uniform Catmull--Rom-Like Quaternion Splines

We have seen how to use
[de Casteljau's algorithm with Slerp](de-casteljau.ipynb)
to create "cubic" Bézier-like quaternion curve segments.

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

However, if we only have a sequence of quaternions to be interpolated
and no additional Bézier control quaternions are provided,
it would be great if we could compute
the missing control quaternions automatically
from neighboring quaternions.

In the
[notebook about (uniform) Euclidean Catmull--Rom splines](../euclidean/catmull-rom-uniform.ipynb#Using-Bézier-Segments)
we have already seen how this can be done for splines in the Euclidean space:

\begin{equation*}
\boldsymbol{\tilde{x}}_i^{(+)}
= \boldsymbol{x}_i + \frac{\boldsymbol{\dot{x}}_i}{3}
= \boldsymbol{x}_i + \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{6}\\
\boldsymbol{\tilde{x}}_i^{(-)}
= \boldsymbol{x}_i - \frac{\boldsymbol{\dot{x}}_i}{3}
= \boldsymbol{x}_i - \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{6}
\end{equation*}

\begin{equation*}
\boldsymbol{\dot{x}}_i =
\frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2} =
\frac{(\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2} =
\frac{\boldsymbol{x}_i - \boldsymbol{x}_{i-1}}{2} +
\frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_i}{2}
\end{equation*}

\begin{align*}
\tilde{q}_{i}^{(+)}
&=
q_{i,\text{offset}} \; q_i\\
\tilde{q}_{i}^{(-)}
&=
{q_{i,\text{offset}}}^{-1} \; q_i
\end{align*}

[helper.py](helper.py)

In [ ]:
from helper import angles2quat

In [ ]:
q3 = angles2quat(0, 0, 0)
q4 = angles2quat(0, 45, -10)
q5 = angles2quat(90, 0, -90)

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\left(q_{i+1} {q_{i-1}}^{-1}\right)^{\frac{1}{6}}
\end{equation*}

In [ ]:
offset_a = q3.rotation_to(q5)**(1/6)

In order to visualize this situation,
we use a little helper function from 
[helper.py](helper.py):

In [ ]:
from helper import squashed_tangent_space

In [ ]:
# TODO: use (correct) tangent for x-alignment?

In [ ]:
squash = squashed_tangent_space(q4, q5, q3)

We can use this to project the participating quaternions
onto a two-dimensional plane which we can then use to make 2D plots.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
t = np.linspace(0, 1, 100)

In [ ]:
def plot_chords():
    plt.scatter(*squash([q3, q4, q5]).T)
    plt.text(*squash(q3).T, 'q3')
    plt.text(*squash(q4).T, 'q4')
    plt.text(*squash(q5).T, 'q5')
    plt.plot(*squash(slerp(q3, q4, t)).T)
    plt.plot(*squash(slerp(q4, q5, t)).T)

In [ ]:
plot_chords()
plt.scatter(*squash([offset_a * q4]).T)
plt.plot(*squash(slerp(q3, q5, t)).T, '--')
plt.plot(*squash(slerp(q4, offset_a * q4, t)).T, ':')
plt.axis('equal');

\begin{align*}
q_\text{in} &= q_i {q_{i-1}}^{-1}\\
q_\text{out} &= q_{i+1} {q_i}^{-1}
\end{align*}

In [ ]:
q_in = q3.rotation_to(q4)
q_out = q4.rotation_to(q5)

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\left(q_\text{out} q_\text{in} \right)^{\frac{1}{6}}
\end{equation*}

In [ ]:
offset_b = (q_out * q_in)**(1/6)

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\left(q_\text{in} q_\text{out} \right)^{\frac{1}{6}}
\end{equation*}

In [ ]:
offset_c = (q_in * q_out)**(1/6)

In [ ]:
max(map(abs, (offset_b - offset_c).xyzw))

In [ ]:
from splines.quaternion import UnitQuaternion

In [ ]:
plot_chords()
plt.scatter(*squash([offset_b * q4]).T)

plt.plot(*squash(slerp(UnitQuaternion(), q_in, t) * q4).T, '--')
plt.plot(*squash(slerp(UnitQuaternion(), q_out, t) * q_in * q4).T, '--')
plt.plot(*squash(slerp(q4, offset_b * q4, t)).T, ':')

plt.plot(*squash(slerp(UnitQuaternion(), q_in, t) * q5).T, '--')

plt.axis('equal');

In [ ]:
max(map(abs, (offset_b - offset_a).xyzw)), max(map(abs, (offset_c - offset_a).xyzw))

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\left({q_\text{out}}^{\frac{1}{6}} {q_\text{in}}^{\frac{1}{6}} \right)
\end{equation*}

In [ ]:
offset_d = (q_out**(1/6) * q_in**(1/6))

In [ ]:
plot_chords()
plt.scatter(*squash([offset_d * q4]).T)

plt.plot(*squash(slerp(UnitQuaternion(), q_in**(1/6), t) * q4).T, '--')
plt.plot(*squash(slerp(UnitQuaternion(), q_out**(1/6), t) * q_in**(1/6) * q4).T, '--')

plt.plot(*squash(slerp(q4, offset_d * q4, t)).T, ':')

plt.plot(*squash(slerp(UnitQuaternion(), q_in**(1/6), t) * q_out**(1/6) * q4).T, '--')

plt.axis('equal');

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\left({q_\text{out}}^{\frac{1}{6}} {q_\text{in}}^{\frac{1}{6}} \right)
\end{equation*}

In [ ]:
offset_e = (q_in**(1/6) * q_out**(1/6))

In [ ]:
max(map(abs, (offset_e - offset_d).xyzw))

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\left(
\left(
{q_\text{out}}^{\frac{1}{6}}
{q_\text{in}}^{-\frac{1}{6}}
\right)^{\frac{1}{2}}
{q_\text{in}}^{\frac{1}{6}}
\right)^2
= \left(
\left(
{q_\text{in}}^{\frac{1}{6}}
{q_\text{out}}^{-\frac{1}{6}}
\right)^{\frac{1}{2}}
{q_\text{out}}^{\frac{1}{6}}
\right)^2
\end{equation*}

In [ ]:
offset_f = ((q_out**(1/6) * q_in**(-1/6))**(1/2) * q_in**(1/6))**2

In [ ]:
offset_g = ((q_in**(1/6) * q_out**(-1/6))**(1/2) * q_out**(1/6))**2

In [ ]:
max(map(abs, (offset_g - offset_f).xyzw))

In [ ]:
plot_chords()
plt.scatter(*squash([offset_f * q4]).T)

plt.plot(*squash(slerp(q4, q_in**(1/6) * q4, t)).T, '--')

plt.plot(*squash(
    slerp(UnitQuaternion(), (q_out**(1/6) * q_in**(-1/6))**(1/2), t) *
    q_in**(1/6) * q4).T, '--')

plt.plot(*squash(slerp(q4, offset_f * q4, t)).T, ':')

plt.axis('equal');

\begin{equation*}
q_{i,\text{offset}}
\overset{?}{=}
\exp\left(
\frac{\ln(q_\text{in}) + \ln(q_\text{out})}{6}
\right)
\end{equation*}

In [ ]:
offset_h = UnitQuaternion.exp_map((q_in.log_map() + q_out.log_map()) / 6)

In [ ]:
max(map(abs, (offset_h - offset_f).xyzw))

In [ ]:
def offset(q_1, q0, q1):
    q_in = q0 * q_1.inverse()
    q_out = q1 * q0.inverse()
    return UnitQuaternion.exp_map((q_in.log_map() + q_out.log_map()) / 6)

In [ ]:
from splines.quaternion import DeCasteljau, canonicalized

[helper.py](helper.py)

In [ ]:
from helper import animate_rotations, display_animation

In [ ]:
def create_closed_curve(rotations):
    rotations = list(canonicalized(rotations + rotations[:2]))
    control_points = []
    for q_1, q0, q1 in zip(rotations, rotations[1:], rotations[2:]):
        q_offset = offset(q_1, q0, q1)
        control_points.extend([q_offset.inverse() * q0, q0, q0, q_offset * q0])
    control_points = control_points[-2:] + control_points[:-2]
    segments = list(zip(*[iter(control_points)] * 4))
    return DeCasteljau(segments)

In [ ]:
rotations = [
    angles2quat(0, 0, 180),
    angles2quat(0, 45, 90),
    angles2quat(90, 45, 0),
    angles2quat(90, 90, -90),
    angles2quat(180, 0, -180),
    angles2quat(-90, -45, 180),
]

In [ ]:
s = create_closed_curve(rotations)

In [ ]:
times = np.linspace(0, len(rotations), 200, endpoint=False)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 2))

In [ ]:
display_animation(ani, default_mode='loop')